# Parallelization

In [9]:
!pip install timebudget

In [27]:
import os
print('Number of CPUs in the system: {}'.format(os.cpu_count()))

Number of CPUs in the system: 12


In [28]:
import math
import numpy as np
from timebudget import timebudget

iterations_count = round(1e7)

def complex_operation(input_index):
    print("Complex operation. Input index: {:2d}".format(input_index))

    [math.exp(i) * math.sinh(i) for i in [1] * iterations_count]

@timebudget
def run_complex_operations(operation, input):
    for i in input:
        operation(i)

input = range(10)
run_complex_operations(complex_operation, input)

Complex operation. Input index:  0
Complex operation. Input index:  1
Complex operation. Input index:  2
Complex operation. Input index:  3
Complex operation. Input index:  4
Complex operation. Input index:  5
Complex operation. Input index:  6
Complex operation. Input index:  7
Complex operation. Input index:  8
Complex operation. Input index:  9
run_complex_operations took 25.370sec


# Process-Based Parallelism

In [33]:
import math
import numpy as np
from timebudget import timebudget
from multiprocessing import Pool

iterations_count = round(1e7)

def complex_operation(input_index):
    print("Complex operation. Input index: {:2d}\n".format(input_index))
    [math.exp(i) * math.sinh(i) for i in [1] * iterations_count]

@timebudget
def run_complex_operations(operation, input, pool):
    pool.map(operation, input)

processes_count = 10

# if __name__ == '__main__':
processes_pool = Pool(processes_count)
run_complex_operations(complex_operation, range(10), processes_pool)

Complex operation. Input index:  8
Complex operation. Input index:  1
Complex operation. Input index:  2
Complex operation. Input index:  5
Complex operation. Input index:  0
Complex operation. Input index:  3
Complex operation. Input index:  6
Complex operation. Input index:  7
Complex operation. Input index:  4
Complex operation. Input index:  9










run_complex_operations took 6.914sec


(raylet) [2023-11-25 17:37:25,498 E 42339 42339] (raylet) node_manager.cc:3035: 2 Workers (tasks / actors) killed due to memory pressure (OOM), 0 Workers crashed due to other reasons at node (ID: 57246ea15ef728432a5cbf325a103fd18faee4bee80faa10eadb7272, IP: 172.17.0.2) over the last time period. To see more information about the Workers killed on this node, use `ray logs raylet.out -ip 172.17.0.2`
(raylet) 
(raylet) Refer to the documentation on how to address the out of memory issue: https://docs.ray.io/en/latest/ray-core/scheduling/ray-oom-prevention.html. Consider provisioning more memory on this node or reducing task parallelism by requesting more CPUs per task. To adjust the kill threshold, set the environment variable `RAY_memory_usage_threshold` when starting Ray. To disable worker killing, set the environment variable `RAY_memory_monitor_refresh_ms` to zero.


# Specialized Libraries

In [34]:
import math
import numpy as np
from timebudget import timebudget
from multiprocessing import Pool

iterations_count = round(1e7)

def complex_operation(input_index):
    print("Complex operation. Input index: {:2d}".format(input_index))

    [math.exp(i) * math.sinh(i) for i in [1] * iterations_count]

def complex_operation_numpy(input_index):
    print("Complex operation (numpy). Input index: {:2d}".format(input_index))

    data = np.ones(iterations_count)
    np.exp(data) * np.sinh(data)

@timebudget
def run_complex_operations(operation, input, pool):
    pool.map(operation, input)

processes_count = 10

# if __name__ == '__main__':
processes_pool = Pool(processes_count)
print('Without NumPy')
run_complex_operations(complex_operation, range(10), processes_pool)
print('\nNumPy')
run_complex_operations(complex_operation_numpy, range(10), processes_pool) 

Complex operation. Input index:  6Complex operation. Input index:  9Complex operation. Input index:  0Complex operation. Input index:  5Complex operation. Input index:  2Complex operation. Input index:  8Complex operation. Input index:  1Complex operation. Input index:  3Complex operation. Input index:  7Complex operation. Input index:  4











*** SIGTERM received at time=1700933850 on cpu 4 ***
*** SIGTERM received at time=1700933850 on cpu 9 ***
*** SIGTERM received at time=1700933850 on cpu 2 ***
*** SIGTERM received at time=1700933850 on cpu 8 ***
*** SIGTERM received at time=1700933850 on cpu 11 ***
*** SIGTERM received at time=1700933850 on cpu 1 ***
*** SIGTERM received at time=1700933850 on cpu 7 ***
*** SIGTERM received at time=1700933850 on cpu 0 ***
*** SIGTERM received at time=1700933850 on cpu 6 ***
*** SIGTERM received at time=1700933850 on cpu 5 ***
PC: @           0x4d8220  (unknown)  _PyEval_EvalFrameDefault
PC: @           0x4d8220  (unknown)  _PyEval_EvalFrameDefault
PC: @           0x4d8220  (unknown)  _PyEval_EvalFrameDefault
PC: @           0x4d8220  (unknown)  _PyEval_EvalFrameDefault
PC: @           0x4d8220  (unknown)  _PyEval_EvalFrameDefault
    @     0x7f5a640a5420  (unknown)  (unknown)
    @     0x7f5a640a5420  (unknown)  (unknown)
    @     0x7f5a640a5420  (unknown)  (unknown)
    @     0x7f5a64

Without NumPy
run_complex_operations took 6.135sec
Complex operation (numpy). Input index:  0Complex operation (numpy). Input index:  1Complex operation (numpy). Input index:  2Complex operation (numpy). Input index:  6Complex operation (numpy). Input index:  5Complex operation (numpy). Input index:  7Complex operation (numpy). Input index:  3Complex operation (numpy). Input index:  4Complex operation (numpy). Input index:  9Complex operation (numpy). Input index:  8










NumPy
run_complex_operations took 791.87ms


# Ray

In [24]:
!pip install ray

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 MB 5.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 534.8/534.8 kB 7.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.1/220.1 kB 7.9 MB/s eta 0:00:00


In [35]:
import math
import numpy as np
from timebudget import timebudget
import ray

iterations_count = round(1e7)

@ray.remote
def complex_operation(input_index):
    print("Complex operation. Input index: {:2d}".format(input_index))

    [math.exp(i) * math.sinh(i) for i in [1] * iterations_count]

@ray.remote
def complex_operation_numpy(input_index):
    print("Complex operation (numpy). Input index: {:2d}".format(input_index))

    data = np.ones(iterations_count)
    np.exp(data) * np.sinh(data)

@timebudget
def run_complex_operations(operation, input):
    
	ray.get([operation.remote(i) for i in input])

ray.shutdown()
ray.init()

input = range(10)
print('Without NumPy')
run_complex_operations(complex_operation, input)
print('NumPy')
run_complex_operations(complex_operation_numpy, input)

2023-11-25 17:38:22,750	WARNING services.py:1996 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 67059712 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=2.13gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.
2023-11-25 17:38:23,893	INFO worker.py:1673 -- Started a local Ray instance.


Without NumPy
(complex_operation pid=43803) Complex operation. Input index:  0
run_complex_operations took 6.387sec
NumPy
(complex_operation_numpy pid=43799) Complex operation (numpy). Input index:  0
(complex_operation pid=43804) Complex operation. Input index:  9 [repeated 9x across cluster]
run_complex_operations took 823.77ms
